# Trabajar con archivos post-mortem

En **<font color='0066FB'>Asesoftware</font>** existen archivos con la información post-mortem de los proyectos: los documentos de cierre de proyecto. En este documento, en la sección de lecciones aprendidas, se consignan los aprendizajes del proyecto.

La información de cada lección aprendida, contexto, proyecto, etc. se encuentra en un archivo que representa la base de datos de lecciones aprendidas aprobadas por la organización. El siguiente script obtiene en un dataframe de Pandas la información de contexto y lección aprendida.

***
***
# Análisis Post-Mortem

El procesamiento de lenguaje natural (o NLP por sus siglas en inglés) es una rama de conocimiento de la Inteligencia Artificial que, esencialmente, pretende conseguir que una máquina comprenda lo que expresa una persona mediante el uso de una lengua natural.

Para el análisis de los documentos post-mortem se van a seguir los siguientes pasos:

1. Separación de palabras ( _tokenize_ )
2. Limpieza ( _Stem - Lemma_ )
3. _Part-Of-Speech tagging_
4. _Dependency Parsing_
5. Reconocimiento de entidades ( _Named Entity Recognition_ )

Esto para poder llegar a encontrar **relaciones binarias** entre las entidades halladas en el punto 5.

### Librerías necesarias

#### Descargar paquetes

In [ ]:
!conda install -c conda-forge spacy -y

In [ ]:
!python -m spacy download es_core_news_md

In [ ]:
!conda install -c anaconda pandas -y

#### Importar librerías

In [ ]:
import pandas as pd
import spacy

In [ ]:
# Importar vocabulario de Spacy, removiendo del pipeline el NER
nlp = spacy.load('es_core_news_md', disable=['ner'])

In [ ]:
def concat_text(pdSeries):
    pdSeries = pdSeries.str.rstrip('.')
    return pdSeries.str.cat(sep='. ')

In [ ]:
nombre_archivo = '../input/REPOSITORIO_LECCIONES APRENDIDAS.xlsx'

data = pd.read_excel(nombre_archivo, encoding='latin-1', keep_default_na= False, na_values=[""])
data["CONTEX_LECC"] = [concat_text(i[1]) for i in data[['CONTEXTO', 'LECCIONES APRENDIDAS']].iterrows()]
data.CONTEX_LECC = data.CONTEX_LECC.str.replace('\n', ' ').replace('\s+', ' ')

data.head()

En la columna `CONTEX_LECC` están concatenadas las oraciones de contexto y lecciones, y `cont_lecc` (en la siguiente celda) es el texto que se va a analizar con el pipeline de NLP.

In [ ]:
cont_lecc = concat_text(data["CONTEX_LECC"])

En Spacy, el modelo entrenado en español incluye en su _pipeline:_

- Tokenizer
- Lemmatizer
- POS-tagger
- NER

Sin embargo, por los resultados obtenidos en pruebas concluimos que el NER de Spacy necesita ser mejorado para poder usarlo con confianza en este proceso de extracción de información.

En la siguiente celda se extraen los lemmas, el POS-tagging y el dependency parsing de la columna `CONTEX_LECC`, junto con una lista de _dependientes_ de la palabra en el árbol del dependency parsing

In [ ]:
word = []
lemma = []
shape = []
pos = []
istop = []
dep = []
head = []
children = []

nlp_text=nlp(cont_lecc)

for token in nlp_text:
    word.append(token.text)
    lemma.append(token.lemma_)
    shape.append(token.shape_)
    pos.append(token.pos_)
    istop.append(token.is_stop)
    dep.append(token.dep_)
    head.append(token.head.text)
    children.append([child for child in token.children])

    
results = pd.DataFrame({'Word':word, 'Lemma':lemma, 'POS':pos, 'DEP':dep, 'head':head,
                             'children':children, 'Shape':shape, 'is_stop':istop})
results.head()

El objetivo, una vez obtenidas las categorías gramaticales (POS) y las dependencias es extraer entidades y sus relaciones.

Esto se puede lograr mediante:
* Reglas
* Modelos de machine learning no supervisados (clustering o neural networks)

Para extraer reglas, se analizaron las gráficas de dependency parsing y se sacaron diferentes reglas observadas.

In [ ]:
info = []

for possible_subject in nlp_text:
    # si el POS del HEAD de la palabra es VERB y su dependency parsing es nsubj (sujeto nominal)
    if possible_subject.head.pos_ == 'VERB' and possible_subject.dep_ == 'nsubj' :
        children = []
        for child in possible_subject.children:
            # si las ramas son nmod (modificador nominal) y no es espacio
             if child.dep_ in ('nmod') and child.pos_ != 'SPACE': 
                children.append(child)
            
        if children:
            info.append((possible_subject.head.lemma_.lower(),possible_subject.lemma_.lower(),children))
result = pd.DataFrame(info, columns = ['Head' , 'Word', 'Children'])
result.head()

In [ ]:
# !!!
info = []
for possible_subject in nlp_text:
    # Si el POS del HEAD de la palabra es VERB y el POS de la palabra es un sustantivo (PROPN y NOUN) y
    # su dependency parsing es sujeto nominal (nsubj)
    if possible_subject.head.pos_ == 'VERB' and possible_subject.pos_ in ('PROPN','NOUN') and possible_subject.dep_=='nsubj':
        info.append((possible_subject.head,possible_subject,possible_subject.lemma_))
        
result_subj = pd.DataFrame(info, columns = ['Head','Word', 'Lemma'])
result_subj.head()

In [ ]:
info = []
for possible_subject in nlp_text:
    # Si el POS del HEAD de la palabra es VERB y el POS de la palabra es un sustantivo (PROPN y NOUN) y
    # su dependency parsing es sujeto nominal (nsubj)
    if possible_subject.head.pos_ == 'VERB' and possible_subject.pos_ in ('PROPN','NOUN') and possible_subject.dep_=='nsubj':
        children = []
        for child in possible_subject.children:
            # Solo agregar si no es identificado como espacio
            if child.pos_ != 'SPACE':
                children.append(child)
            
        if children:
            info.append((possible_subject.head.lemma_,possible_subject,possible_subject.lemma_.lower(),children))
            
result_subj1 = pd.DataFrame(info, columns = ['Head' , 'Word', 'Lemma', 'Children'])

In [ ]:
info = []
for possible_subject in nlp_text:
    if possible_subject.pos_ == 'VERB' and possible_subject.dep_=='nsubj':
        children = []
        for child in possible_subject.children:
            # Solo agregar si no es identificado como espacio
            if child.pos_ != 'SPACE':
                children.append(child)
            
        if children:
            info.append((possible_subject.head.lemma_,possible_subject,possible_subject.lemma_.lower(),children))
            
result_subj2 = pd.DataFrame(info, columns = ['Head' , 'Word', 'Lemma', 'Children'])
result_subj2.head()

In [ ]:
print(result_subj2.Lemma.value_counts().nlargest(10, keep='all'))

In [ ]:
from __future__ import unicode_literals
import textacy
from collections import defaultdict

###
# Patrón para extraer información de un texto basado en reglas(con expresiones regulares basados en token).
###

patron = r'<PROPN>+ (<PUNCT|CCONJ> <PUNCT|CCONJ>? <PROPN>+)*'
param = []
i = 0
while i < len(data["CONTEX_LECC"]):
    lists_ = []
    sent = nlp(data["CONTEX_LECC"].iloc[i])
    doc = textacy.make_spacy_doc(sent, lang='es_core_news_md')
    lists_ = textacy.extract.pos_regex_matches(doc, patron)
    for item in lists_:
        if len(item) != 0:
            param.append(item.text.lower())
    i +=1

j=0
aux = defaultdict(list)
for index, item in enumerate(param):
    aux[item].append(index)

result = {item: len(indexs) for item, indexs in aux.items() if len(indexs) >= 1}
key = list(result.keys())
key.sort()

In [ ]:
#Normalizar los datos del diccionario de entidades
key = list(result.keys())
print(len(key))
for item in key:
    i = 0
    key_new = []
    key_new = key
    key_new.remove(item)
    while i < len(key) - 1:        
#         print(len(key_new))
        i += 1

Combinamos la regla de entidades con la regla de relaciones en la siguiente celda

In [ ]:
info = []

for possible_subject in nlp_text:
    # si el POS del HEAD de la palabra es VERB y su dependency parsing es nsubj (sujeto nominal)
    if possible_subject.head.pos_ == 'VERB' and possible_subject.dep_ == 'nsubj' :
        children = []
        for child in possible_subject.children:
            # si las ramas son nmod (modificador nominal) y no es espacio
            if str(child).lower() in key:
                 if child.dep_ in ('nmod') and child.pos_ != 'SPACE':
                        info.append((possible_subject.text.lower(),possible_subject.head.lemma_.lower(),child.text.lower()))

result_ent = pd.DataFrame(info, columns = ['Word', 'Head', 'Children'])
result_ent

In [ ]:
result_ent[result_ent.Children == 'asesoftware']

In [ ]:
info = []
for possible_subject in nlp_text:
    if possible_subject.head.pos_ == 'VERB' and possible_subject.pos_ in ('PROPN','NOUN'):
        children = []
        for child in possible_subject.children:
            # Solo agregar si no es identificado como espacio
            if str(child).lower() in key:
                if child.pos_ != 'SPACE':
                    info.append((possible_subject.text.lower(),possible_subject.head.lemma_.lower(),child.text.lower()))
                                    
result_ent1 = pd.DataFrame(info, columns = ['Word', 'Head', 'Children'])
result_ent1

In [ ]:
result_ent1[result_ent1.Children == 'proyecto']